In [1]:
%pip install lightfm

In [3]:
import numpy as np
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
import pandas as pd
from scipy.sparse import csr_matrix
import dask.dataframe as dd
from joblib import dump, load
BASE_PATH = 'drive/MyDrive/'

In [4]:
# Загрузка и обработка данных
users = pd.read_csv(BASE_PATH + "small_player_starts_train.csv")
#users = users.sample(frac=0.5, random_state=42)  # Обрезаем датасет (только для демонстрации)


users['user_id_num'] = users['user_id'].str.split('_').str.get(1).astype(int)
users['item_id_num'] = users['item_id'].str.split('_').str.get(1).astype(int)

# Создаем разреженную матрицу
user_ids = users['user_id_num'].values
item_ids = users['item_id_num'].values

# Предполагая, что у вас есть столбец с рейтингами/взаимодействиями
# Если его нет, можно просто использовать np.ones_like(user_ids)
ratings = np.ones_like(user_ids)

data_crosstab = csr_matrix((ratings, (user_ids, item_ids)))

In [5]:
# У вас уже есть разреженная матрица взаимодействий, поэтому нет необходимости создавать ее снова.
interactions_csr = data_crosstab

# Предположим, что у вас есть список уникальных user_id и item_id, которые вы использовали при создании data_crosstab
unique_user_ids = np.unique(users['user_id_num'])
unique_item_ids = np.unique(users['item_id_num'])

# Создание отображений
user_id_mapping = {real_id: idx for idx, real_id in enumerate(unique_user_ids)}
item_id_mapping = {real_id: idx for idx, real_id in enumerate(unique_item_ids)}

# Предварительная генерация массива для быстрого поиска
item_lookup_array = np.empty(interactions_csr.shape[1], dtype=np.int64)
for real_id, idx in item_id_mapping.items():
    item_lookup_array[idx] = real_id

In [ ]:
# Обучение модели
model = LightFM(no_components=20, loss='warp', k=5)
model.fit(interactions_csr, epochs=10, num_threads=4)

In [ ]:
dump(model, 'lightfm_model.joblib')

['lightfm_model.joblib']

In [ ]:
model = load('lightfm_model.joblib')

In [ ]:
# Оценка модели
train_precision = precision_at_k(model, interactions_csr, k=5).mean()
print(f'Train precision at k=5: {train_precision}')

In [ ]:
#videos = pd.read_parquet(BASE_PATH + "videos.parquet")
#videos = dd.read_parquet(BASE_PATH + 'videos.parquet', engine='pyarrow')

#videos_t = videos.sort_values(by='channel_sub', ascending=False)
#top_videos = videos_t['video_id'].values[:100]

# Функция рекомендации
def recommend_for_user_fast(real_user_id):
    internal_user_id = user_id_mapping.get(real_user_id)
    if internal_user_id is None:
        print(f"User {real_user_id} is not in the data.")
        #top_videos = np.random.choice(top_videos, size=10, replace=False)
        #return top_videos
        return []

    n_users, n_items = interactions_csr.shape
    scores = model.predict(internal_user_id, np.arange(n_items))
    top_items = np.argsort(-scores)[:5]

    top_real_item_ids = item_lookup_array[top_items]

    return top_real_item_ids

In [ ]:
users.head()

,date,user_id,item_id,watch_time,is_autorized,user_id_num,item_id_num
0,2023-08-21 15:53:30+03:00,user_7941459,video_1535803,22,0,7941459,1535803
1,2023-08-21 16:17:58+03:00,user_17893165,video_796847,374,0,17893165,796847
2,2023-08-21 21:18:12+03:00,user_25219604,video_1905049,11,0,25219604,1905049
4,2023-08-21 17:24:32+03:00,user_25483180,video_500499,20,0,25483180,500499
5,2023-08-21 08:33:26+03:00,user_3530167,video_1508131,21,0,3530167,1508131


In [ ]:
real_user_id = 17893165  # Замените на реальный user_id из вашего датасета
recommended_item_ids = recommend_for_user_fast(real_user_id)
print(f"Recommended items for user {real_user_id}: {recommended_item_ids}")

In [ ]:
# Открытие файла sample_submission.csv
submission = pd.read_csv(BASE_PATH + "sample_submission.csv")

# Создаем пустой список для хранения рекомендаций
recommendations = []

# В цикле берем значения user_id из submission и вызываем функцию recommend_for_user_fast
for user_id in submission['user_id']:
    uid = int(user_id.split('_')[1].strip())

    recs = recommend_for_user_fast(uid)
    # Преобразуем рекомендации в строку, разделенную пробелами (или другим разделителем по вашему выбору)
    recs_str = " ".join(map(str, recs))
    recommendations.append(recs_str)

# Записываем результат в колонку recs в submission
submission['recs'] = recommendations

# Сохраняем измененный DataFrame в новый CSV-файл
submission.to_csv("new_sample_submission.csv", index=False)

UsageError: Line magic function `%gzip` not found.
